<center>

# Web Intelligence

# Who will win the Tennis Australian Open 2020?

## 00 Dataset Download and Examination


### Riccardo Spolaor (864877)

</center>

## Index

1. [Scaricare il dataset e rendere i dati facilmente accessibili](#s_1)
1. Section 2

<a id="s_1"></a>

## Scaricare il dataset e rendere i dati facilmente accessibili

Il primo step consiste nello scaricare i vari dataset, importarli in oggetti DataFrame della libreria pandas ed unirli in un solo DataFrame.

Tengo traccia dei differenti dataset scaricati assegnando ad ognuno una feature ***csvID***.

Ignoro i dati dell'anno 2000 in quanto sono privi di dati riguardo alle scommesse.

In [1]:
import urllib.request, zipfile, pandas
import numpy as np

In [2]:
def get_file(url):
    filehandle, _ = urllib.request.urlretrieve(url)
    zip_file_obj = zipfile.ZipFile(filehandle, 'r')
    file = zip_file_obj.open(zip_file_obj.namelist()[0])
    return file

dataframe = pandas.DataFrame()

for date in range(2010,2019+1):
    file = get_file('http://tennis-data.co.uk/' + str(date) + '/' + str(date) + '.zip')
    dataframe = dataframe.append(pandas.read_excel(file), sort = False, ignore_index = True)


dataframe.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,LBW,LBL,PSW,PSL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
0,1,Brisbane,Brisbane International,2010-01-04,ATP250,Outdoor,Hard,1st Round,3,Gasquet R.,...,1.533,2.375,1.526,2.740,1.500,2.500,NaN,NaN,NaN,NaN
1,1,Brisbane,Brisbane International,2010-01-04,ATP250,Outdoor,Hard,1st Round,3,Odesnik W.,...,2.250,1.571,2.140,1.813,2.250,1.571,NaN,NaN,NaN,NaN
2,1,Brisbane,Brisbane International,2010-01-04,ATP250,Outdoor,Hard,1st Round,3,Gicquel M.,...,1.667,2.100,1.676,2.360,1.615,2.200,NaN,NaN,NaN,NaN
3,1,Brisbane,Brisbane International,2010-01-04,ATP250,Outdoor,Hard,1st Round,3,Falla A.,...,2.500,1.500,2.580,1.581,2.500,1.500,NaN,NaN,NaN,NaN
4,1,Brisbane,Brisbane International,2010-01-04,ATP250,Outdoor,Hard,1st Round,3,Levy H.,...,2.625,1.444,2.170,1.787,3.000,1.333,NaN,NaN,NaN,NaN


In [3]:
dataframe.columns.values

array(['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court',
       'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank',
       'WPts', 'LPts', 'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4',
       'W5', 'L5', 'Wsets', 'Lsets', 'Comment', 'B365W', 'B365L', 'EXW',
       'EXL', 'LBW', 'LBL', 'PSW', 'PSL', 'SJW', 'SJL', 'MaxW', 'MaxL',
       'AvgW', 'AvgL'], dtype=object)

In [4]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26328 entries, 0 to 26327
Data columns (total 42 columns):
ATP           26328 non-null int64
Location      26328 non-null object
Tournament    26328 non-null object
Date          26328 non-null datetime64[ns]
Series        26328 non-null object
Court         26328 non-null object
Surface       26328 non-null object
Round         26328 non-null object
Best of       26328 non-null int64
Winner        26328 non-null object
Loser         26328 non-null object
WRank         26317 non-null float64
LRank         26276 non-null float64
WPts          26319 non-null float64
LPts          26277 non-null float64
W1            26161 non-null float64
L1            26163 non-null float64
W2            25908 non-null float64
L2            25908 non-null float64
W3            12377 non-null float64
L3            12377 non-null float64
W4            2444 non-null float64
L4            2444 non-null float64
W5            917 non-null float64
L5          

### Correzione dei tipi assegnati in modo scorretto
Controllo se alcune feature hanno assegnati tipi differenti da quelli attesi e correggo eventuali valori errati

In [5]:
print('Feature di tipo Object:\n', dataframe.select_dtypes(exclude= np.number).columns.tolist())

dataframe.select_dtypes(exclude= np.number).head()

Feature di tipo Object:
 ['Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface', 'Round', 'Winner', 'Loser', 'Comment', 'EXW']


,Location,Tournament,Date,Series,Court,Surface,Round,Winner,Loser,Comment,EXW
0,Brisbane,Brisbane International,2010-01-04,ATP250,Outdoor,Hard,1st Round,Gasquet R.,Nieminen J.,Completed,1.48
1,Brisbane,Brisbane International,2010-01-04,ATP250,Outdoor,Hard,1st Round,Odesnik W.,Clement A.,Completed,2.2
2,Brisbane,Brisbane International,2010-01-04,ATP250,Outdoor,Hard,1st Round,Gicquel M.,Petzschner P.,Completed,1.65
3,Brisbane,Brisbane International,2010-01-04,ATP250,Outdoor,Hard,1st Round,Falla A.,Chardy J.,Completed,2.58
4,Brisbane,Brisbane International,2010-01-04,ATP250,Outdoor,Hard,1st Round,Levy H.,Llodra M.,Completed,2.6


***EXW*** presenta valori numerici, ma è categorizzato come *Object*.

Controllo quali sono i valori non numerici che non permettono il corretto casting a float e procedo alla loro correzione

In [6]:
import numbers

def checkValues(attribute_names):
    for a in attribute_names:
        print('"'+a+'": ', 'Wrong Values -> ' , set([x for x in dataframe[a] if not isinstance(x, numbers.Number)]))
    
checkValues(['EXW'])

"EXW":  Wrong Values ->  {'2.,3'}


In [7]:
import re

def correctTypos(attribute_names):
    for a in attribute_names:
        wrong_values = set([x for x in dataframe[a] if not isinstance(x, numbers.Number)])
        indexes = dataframe[dataframe[a].isin(wrong_values)].index.values
        dataframe.iloc[indexes]
        for i in indexes:
            dataframe.iloc[i,dataframe.columns.get_loc(a)] = float(re.sub("[^0-9^.]", "", str(dataframe.iloc[i][a])))            
        dataframe[a] = dataframe[a].astype(float)

In [8]:
print('EXW has typos.')
print('It is possible to correct these typos and assign "float" as the features type :\n')

correctTypos(['EXW'])
checkValues(['EXW'])

EXW has typos.
It is possible to correct these typos and assign "float" as the features type :

"EXW":  Wrong Values ->  set()


In [9]:
dataframe.to_csv('./datasets/dataframe00.csv', index = False)